In [1]:
from gurobipy import *

## Reading a Graph

We must first provide an efficient way to read in a Graph from a .txt file. Graphs can be represented in a number of different ways but for this workshop we consider the style as shown in jazz.graph.txt. The first line of this file has two numbers, the first reprents the number of vertices in $G$, the second represents the number edges. Each line after that represents an edge, defined by its two end points which are vertices in $G$.

I have provided the code to do this below

In [2]:
def read(inputfile):
    f = open(inputfile, 'r') # Open .txt file
    line = f.readline() # Read the first line, returns string
    fields = str.split(line) # Split the line, returns list
    n = int(fields[0]) # Number of vertices in G
    m = int(fields[1]) # Number of edges in G
    adjLists = [] # Initialize adjacency list

    for i in range(0, n):
        adjLists.append([])
    
    # Fill adjacency list
    for line in f:
        fields = line.split(' ')
        i = int(fields[0])
        j = int(fields[1])
        adjLists[i].append(j)
        adjLists[j].append(i)
    f.close # Close .txt file once finished reading
    return n, m, adjLists

In [3]:
n, m, adjLists = read("../dat/jazz.graph.txt")

In [6]:
adjLists[0:3]

[[7,
  23,
  34,
  41,
  45,
  59,
  73,
  77,
  80,
  94,
  97,
  98,
  99,
  100,
  102,
  103,
  107,
  130,
  131,
  153,
  158,
  167,
  170],
 [13, 53, 149],
 [114, 152, 166, 171]]

## Max Weighted Clique

Recall the formulation:

$$ \max \quad\sum_{i \in V} \bar{x}_i y_i \hspace{2.75cm}$$
$$ \text{s.t.} \quad y_i + y_j \leq 1 \quad \lbrace i,j \rbrace \notin E$$
$$  y_i \in \lbrace 0,1 \rbrace \quad i \in V $$


Since we have not solved the LP relaxation for vertex packing yet $\bar{x}_i= 0$ for all $i \in V$ for now. We will update the objective coefficient for $y_i$ in our callback function  



In [7]:
MWC_model = Model('MaxWeightClique')
MWC_model.Params.OutputFlag = 0 
y = {}
for i in range(n):
    y[i] = MWC_model.addVar(vtype=GRB.BINARY, obj = 0, name="y_{}".format(i))

MWC_model.update()
MWC_model.modelSense = GRB.MAXIMIZE

for i in range(n):
    for j in range(i+1, n):
        if (j not in adjLists[i]):
            MWC_model.addConstr(y[i] + y[j] <= 1)

Academic license - for non-commercial use only


## Vertex Packing Formulation

Recall the vertex packing formulation:

$$\max \quad \sum_{i \in V} x_i \hspace{3cm}$$
$$\text{s.t.} \quad x_i + x_j \leq 1 \quad \lbrace i,j \rbrace \in E$$
$$ x_i \in \lbrace 0,1 \rbrace \quad i \in V$$

### IMPORTANT: Model Parameters

In order to use user cuts in our model we must set the model parameter PreCrush equal to 1

```python
VP_model.Params.PreCrush = 1
```

It turns out that the developers of Gurobi have their own implementation of clique cuts that is much better than the procedure we have discussed here. In addition Gurobi has some very powerful presolve algorithms that exploit the problem structure to make the model much smaller and easier to solve. Without disabling these, Gurobi will add all of their cuts and solve their problem to optimality before we even get a chance to use our cuts! The following code will prevent Gurobi from adding their clique cuts and using their presolve algorithms.

```python
VP_model.Params.CliqueCuts = 0
VP_model.Params.Presolve = 0
```

In [23]:
VP_model = Model('VertexPacking') 
VP_model.Params.PreCrush = 1 # Allow user cuts
VP_model.Params.CliqueCuts = 0 # Prevent Gurobi from adding their clique cuts
VP_model.Params.Presolve = 0 
x = {}
for i in range(n):
    x[i] = VP_model.addVar(vtype = GRB.BINARY, obj = 1, name="x_{}".format(i))
    
VP_model.update()
VP_model.modelSense = GRB.MAXIMIZE

for i in range(n):
    for j in adjLists[i]:
        if (i < j):
            VP_model.addConstr(x[i] + x[j] <= 1)

Changed value of parameter PreCrush to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter CliqueCuts to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1


## Load Data in VP Model

Since our callback can only take two arguements, we need load our variables, parameters and MWC model into our vertex packing model in order to access them during the callback

In [24]:
VP_model._x = x
VP_model._n = n
VP_model._MWC = MWC_model
VP_model._y = y

## Separation Algorithm

Our callback function CliqueCuts may only have two arguments: model and where. The model will be our vertex packing model. We use "where" in order to query the status of our model. The statement 


```python
if (where == GRB.Callback.MIPNODE)
```
indicates that we want query the model once we open a MIP node in the branch and bound tree. However, we are only interested in nodes where we can get an optimal LP relaxation (as opposed to infeasible). The statement

```python
status = VP_model.cbGet(GRB.Callback.MIPNODE_STATUS)
```
queries the status of our current node. Since we will use our callback to add cuts based on the solution LP relaxation, we can ensure that a feasible solution is found via

```python
if status == GRB.OPTIMAL
```

In [25]:
def CliqueCuts(VP_model, where):
    if (where == GRB.Callback.MIPNODE):
        status = VP_model.cbGet(GRB.Callback.MIPNODE_STATUS)
        if status == GRB.OPTIMAL:
            x_sol = VP_model.cbGetNodeRel(VP_model._x)
            for i in range(n):
                VP_model._y[i].Obj = x_sol[i]
            VP_model._MWC.optimize()
            
            print('Max weighted clique: {}'.format(VP_model._MWC.objval))
            for i in range(n):
                if (y[i].x > 0.5):
                    print("{} ".format(i), end='')
            print("\n")
            
            if VP_model._MWC.objval > 1:
                VP_model.cbCut(quicksum(VP_model._x[i]*y[i].x for i in range(VP_model._n)) <= 1)
            
            

In [26]:
VP_model.optimize()

Optimize a model with 2742 rows, 198 columns and 5484 nonzeros
Variable types: 0 continuous, 198 integer (198 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 34.0000000
Variable types: 0 continuous, 198 integer (198 binary)

Root relaxation: objective 9.900000e+01, 102 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   99.00000    0  188   34.00000   99.00000   191%     -    0s
     0     0   87.00000    0  164   34.00000   87.00000   156%     -    0s
H    0     0                      35.0000000   87.00000   149%     -    0s
     0     0   86.00000    0  162   35.00000   86.00000   146%     -    0s
     0     0   79.41667    0  152   35.00000   79.41667   127%     -    0s
     0     0   78.5000